In [26]:
import functions as fn
fn = reload(fn)

In [7]:
import pyfits as pf
import sys, os, shutil

nfiles = 0
# rootDir = '/nfs/lsst2/photocalData/data/monocam/sync_sorted/1m3_bad_data/'
# dest_dir = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3_bad_data/'

# rootDir = '/nfs/lsst2/photocalData/data/monocam/sync_sorted/1m3/'
# dest_dir = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/'

rootDir = '/nfs/lsst2/photocalData/data/monocam/sync_sorted/61inch/'
dest_dir = '/nfs/lsst2/photocalData/data/monocam/sanitised/61inch/'

# rootDir = '/nfs/lsst2/photocalData/data/monocam/sync_sorted/61inch_bad_data/'
# dest_dir = '/nfs/lsst2/photocalData/data/monocam/sanitised/61inch_bad_data/'
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        filename = os.path.join(dirName,fname)
        if filename.find('.eko')!=-1:
            print 'Skipped file %s'%filename
            continue
            
        f = pf.open(filename)
        primaryHeader = f[0].header
        date = primaryHeader['DATE-OBS'].split('T')[0]
        f.close()
        
        dest_filename = dest_dir + date + '_' + fname
        if os.path.exists(dest_filename):
            print 'Error - nearly overwrote file:'
            print 'Source file: %s'%filename
            print 'Destination file: %s'%dest_filename
            shutil.move(filename, filename + '_01')

        else:
#             print 'would have moved %s to %s'%(filename, dest_filename)
            shutil.move(filename, dest_filename)


    
        nfiles += 1
print 'Finished, moved %s files'%nfiles

Skipped file /nfs/lsst2/photocalData/data/monocam/sync_sorted/61inch/2016_05_05/y16d126.eko
Skipped file /nfs/lsst2/photocalData/data/monocam/sync_sorted/61inch/2016_05_04/y16d125.eko
Finished, moved 576 files


In [ ]:
import pyfits as pf
import sys, os, shutil

nfiles = 0
rootDir = '/nfs/lsst2/photocalData/data/monocam/sync_to_suction/'
for dirName, subdirList, fileList in os.walk(rootDir):
    for fname in fileList:
        filename = os.path.join(dirName,fname)
        if filename.find('.eko')!=-1:
            print 'Found mystery file %s'%filename
            dest_filename = '/nfs/lsst2/photocalData/data/monocam/sanitised/misc_metadata/' + fname
#             print 'would have moved %s to %s'%(filename, dest_filename)
            shutil.move(filename, dest_filename)
            continue
            
        f = pf.open(filename)
        primaryHeader = f[0].header
        origin = primaryHeader['ORIGIN']
        if origin!='USNO-FS':
            print 'Skipping %s'%filename
            f.close()
            continue
            
        telescope  = primaryHeader['TELESCOP']
        date = primaryHeader['DATE-OBS']
        f.close()
        if telescope == 'NO13':
            dest_filename = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3_metadata/' + date + '_' + fname
#             print 'would have moved %s to %s'%(filename, dest_filename)
            shutil.move(filename, dest_filename)
        elif telescope == 'NO61':
            dest_filename = '/nfs/lsst2/photocalData/data/monocam/sanitised/61inch_metadata/' + date + '_' + fname
#             print 'would have moved %s to %s'%(filename, dest_filename)
            shutil.move(filename, dest_filename)
        else:
            print 'Found mystery file %s'%filename
            dest_filename = '/nfs/lsst2/photocalData/data/monocam/sanitised/misc_metadata/' + date + '_' + fname
#             print 'would have moved %s to %s'%(filename, dest_filename)
            shutil.move(filename, dest_filename)
        nfiles += 1
        print 'copied %s'%fname
        
print 'Finished, moved %s files'%nfiles

In [19]:
dirName = '/nfs/lsst2/photocalData/data/monocam/sanitised/61inch/'
import filecmp, glob

filelist = glob.glob(dirName + '*_01')

n_comp = 0
for filename in filelist:
    cmp_file = filename[:-3]
    n_comp += 1
    if not filecmp.cmp(filename, cmp_file):
        print filename
#     os.remove(filename)

print "Compared %s files"%n_comp


Compared 576 files


In [119]:
pairings = {}


In [120]:
data_path = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/'
md_path = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/'

for fname in os.listdir(data_path):
    if fname[0]=='.': continue
    if fname == 'no_matching_md': continue
    if fname == 'calibs': continue

    filename = data_path + fname
    print filename
    f = pf.open(filename)
    primaryHeader = f[0].header
    header_number = primaryHeader['NUMBER']
    date = primaryHeader['DATE-OBS'].split('T')[0]
    time = primaryHeader['DATE-OBS'].split('T')[1]
    f.close()

#     print filename
#     print date
#     print time
#     print header_number
    glob_string = md_path + date + '_*.' + str(header_number).zfill(3)
#     print glob_string
    mdfiles = glob.glob(glob_string)
    if len(mdfiles) != 1:
        print 'Found %s md files for %s'%(len(mdfiles),filename)
        problem_dest = data_path + 'no_matching_md/' + fname
        shutil.move(filename, problem_dest)
        print
        print
        print
        print
        print

    else:
        pairings[filename] = mdfiles[0]
print 'finished'
print 'Found %s pairings'%len(pairings)


/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-10_HD163466_grat_01.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_BD+02_3375_i_05.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_CAblank5_y4_26.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_CAblank5_y4_45.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-11_M92_i_02.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-10_HD163466_y3_09.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-11_CAblank5_y4_01.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_HD163466_grating_05.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-11_CAblank5_z_01.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_CAblank5_y4_28.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_BD+60_1753_r_09.fits
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1

In [114]:

fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sanitised/61inch/61inch_bad_data/2016-05-02_20160502224139-813-RA.fits')
print '----'
fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3_metadata/2016-05-10_g16d131.208', max_lines=15)


SIMPLE True
BITPIX 16
NAXIS 0
EXTEND True
HISTORY Created with RTS2 version 0.9.4 build on Apr 12 2013 14:04:47.
CTIME 1462228899
USEC 813829
JD 2457511.44559
DATE-OBS 2016-05-02T22:41:39.813
EXPOSURE 0.0
EXPTIME 0.0
INSTRUME 112-04
TELESCOP LSST lab
ORIGIN BNL
SCRIPREP 0
SCRIPT D 0 D 1 D 5 D 10 D 15 D 30 D 60 D 120
SCR_COMM 
COMM_NUM 0
CCD_TYPE 
CCD_SER 
BINNING 1x1
BINX 1
BINY 1
SHUTTER 1
DATA_CHANNELS 16
CHAN 16
CHAN1 True
CHAN2 True
CHAN3 True
CHAN4 True
CHAN5 True
CHAN6 True
CHAN7 True
CHAN8 True
CHAN9 True
CHAN10 True
CHAN11 True
CHAN12 True
CHAN13 True
CHAN14 True
CHAN15 True
CHAN16 True
CONFIG /home/ccdtest/reflex/Timing/e2v250/545kHz_20130531a.rcf
RAW_MODE False
baud_rate 9600
int.error_code 0
int.error_source 0
int.error_line 0
int.status_index 4732
int.power 3
int.backplane_type 268566784
int.interface_type 537067776
int.powerA_type 805634304
int.powerB_type 1074004224
int.daughter1_type 1359085824
int.daughter2_type 0
int.daughter3_type 1879245056
int.daughter4_type 1879245

IOError: [Errno 2] No such file or directory: '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3_metadata/2016-05-10_g16d131.208'

In [121]:
deltas = []

for data_file in pairings:
    md_file = pairings[data_file]
#     print datafile
#     print md_file
    
    d_file = pf.open(data_file)
    primaryHeader = d_file[0].header
    data_date = primaryHeader['DATE-OBS'].split('T')[0]
    data_time = primaryHeader['DATE-OBS'].split('T')[1]
    d_file.close()
    
    m_file = pf.open(md_file)
    primaryHeader = m_file[0].header
    md_date = primaryHeader['DATE'].split('T')[0]
    md_time = primaryHeader['DATE'].split('T')[1]
    m_file.close()

    dt = (TimeToInt(md_time) - TimeToInt(data_time))/1000.
    print '%s \t for %s'%(dt,data_file)
    deltas.append(dt)
    


56.607 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_BD+372346_r_01.fits
65.558 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-11_BD+37_2346_03.fits
22.382 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_BD+02_3375_y4_12.fits
122.924 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_BD+60_1753_u_12.fits
32.951 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-10_SZCVn_y3_01.fits
6.866 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_HD158485_grat_123.fits
3.158 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_HD158485_grat_131.fits
61.386 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_CAblank5_y4_88.fits
7.342 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_HD163466_grating_24.fits
70.847 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_CAblank5_y4_57.fits
7.

In [76]:
def TimeToInt(time_as_string):
    hours = int(time_as_string.split(':')[0])
    mins = int(time_as_string.split(':')[1])
    secs = float(time_as_string.split(':')[2])
    return int((hours *3600*1000) + (mins*60*1000) + (secs*1000))


4207339

In [92]:
deltas = []

for filename in os.listdir('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/'):
    if filename == 'calibs':continue
    if filename == 'no_matching_md':continue
    data_file = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/' + filename

    md_file = pairings[data_file]
#     print datafile
#     print md_file
    
    d_file = pf.open(data_file)
    primaryHeader = d_file[0].header
    data_date = primaryHeader['DATE-OBS'].split('T')[0]
    data_time = primaryHeader['DATE-OBS'].split('T')[1]
    d_file.close()
    
    m_file = pf.open(md_file)
    primaryHeader = m_file[0].header
    md_date = primaryHeader['DATE'].split('T')[0]
    md_time = primaryHeader['DATE'].split('T')[1]
    m_file.close()

    dt = (TimeToInt(md_time) - TimeToInt(data_time))/1000.
    print '%s \t for %s'%(dt,data_file)
    deltas.append(dt)
    


4.418 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-10_HD163466_grat_01.fits
8.025 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-12_BD+02_3375_i_05.fits
62.638 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-12_CAblank5_y4_26.fits
63.046 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-12_CAblank5_y4_45.fits
122.348 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-11_M92_i_02.fits
5.125 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-10_HD163466_y3_09.fits
303.353 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-11_CAblank5_y4_01.fits
11.865 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-12_HD163466_grating_05.fits
303.172 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-11_CAblank5_z_01.fits
62.4 	 for /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/2016-05-12_CAblank5_y4_28.fits
7.068 	 for /nfs/lsst2/photocalData/data/monocam/sa

In [195]:
n_files = 0
for data_file in pairings:
    md_file = pairings[data_file]
    
    d_file = pf.open(data_file, mode='update')
    m_file = pf.open(md_file)

    primaryHeader = d_file[0].header
    md_primaryHeader = m_file[0].header

#     primaryHeader['DATE-OBS'] = md_primaryHeader['DATE']
#     primaryHeader['DATE'] = md_primaryHeader['DATE']
    just_date = md_primaryHeader['DATE-OBS']
    just_UTC = md_primaryHeader['UTC']
    new_date_obs = just_date + 'T' + just_UTC
    primaryHeader['DATE-OBS'] = new_date_obs
    
    d_file.flush()
    d_file.close()
    m_file.close()
#     break
    n_files += 1
    if n_files %100 ==0: print 'Fixed %s files'%n_files

print 'Finished fixing %s files'%n_files


Fixed 100 files
Fixed 200 files
Fixed 300 files
Fixed 400 files
Fixed 500 files
Fixed 600 files
Fixed 700 files
Fixed 800 files
Fixed 900 files
Fixed 1000 files
Finished fixing 1000 files


In [190]:
print len(pairings)

1000


In [115]:
print len(os.listdir('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/'))

1366


In [122]:
print pairings.keys()[0], pairings[pairings.keys()[0]]

/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-12_BD+372346_r_01.fits /nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/2016-05-12_g16d133.249


In [124]:

fn.printHeaderData(pairings.keys()[0], max_lines=20)
print '----'
print pairings[pairings.keys()[0]]
fn.printHeaderData(pairings[pairings.keys()[0]], max_lines=20)


SIMPLE True
BITPIX 16
NAXIS 0
EXTEND True
HISTORY Created with RTS2 version 0.9.4 build on May  6 2016 18:01:35.
CTIME 1463036411
USEC 393998
JD 2457520.79179
DATE-OBS 2016-05-12T07:00:11.393
EXPOSURE 7.0
EXPTIME 7.0
INSTRUME 113-03
TELESCOP USNO 1.3m
ORIGIN BNL
NUMBER 249
SCRIPREP 0
SCRIPT for 3 { E 7 }
SCR_COMM 
COMM_NUM 0
CCD_TYPE 
----
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/2016-05-12_g16d133.249
SIMPLE True
BITPIX 16
NAXIS 2
NAXIS1 100
NAXIS2 100
DATE 2016-05-12T07:01:08
ORIGIN USNO-FS
TIMESYS UTC
DATE-OBS 2016-05-12
TELESCOP NO13
INSTRUME ashutter
OBSERVER mam
OBJECT BD+37_2346
IMAGETYP OBJECT
FILTER SDSSR
RA 12:59:47.08
DEC +36:58:32.4
EQUINOX 2000.0
RAENC 13:00:35.29
DECENC +36:53:24.7


In [126]:

fn.printHeaderData(pairings.keys()[0], max_lines=20)
print '----'
print pairings[pairings.keys()[0]]
fn.printHeaderData(pairings[pairings.keys()[0]], max_lines=20)


SIMPLE True
BITPIX 16
NAXIS 0
EXTEND True
HISTORY Created with RTS2 version 0.9.4 build on May  6 2016 18:01:35.
CTIME 1463036411
USEC 393998
JD 2457520.79179
DATE-OBS 2016-05-12T07:01:08
EXPOSURE 7.0
EXPTIME 7.0
INSTRUME 113-03
TELESCOP USNO 1.3m
ORIGIN BNL
NUMBER 249
SCRIPREP 0
SCRIPT for 3 { E 7 }
SCR_COMM 
COMM_NUM 0
CCD_TYPE 
----
/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/2016-05-12_g16d133.249
SIMPLE True
BITPIX 16
NAXIS 2
NAXIS1 100
NAXIS2 100
DATE 2016-05-12T07:01:08
ORIGIN USNO-FS
TIMESYS UTC
DATE-OBS 2016-05-12
TELESCOP NO13
INSTRUME ashutter
OBSERVER mam
OBJECT BD+37_2346
IMAGETYP OBJECT
FILTER SDSSR
RA 12:59:47.08
DEC +36:58:32.4
EQUINOX 2000.0
RAENC 13:00:35.29
DECENC +36:53:24.7


In [130]:

fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/calibs/2016-05-10_bias_01.fits', max_lines=999)
# print '----'
# print pairings[pairings.keys()[0]]
# fn.printHeaderData(pairings[pairings.keys()[0]], max_lines=20)

SIMPLE True
BITPIX 16
NAXIS 0
EXTEND True
HISTORY Created with RTS2 version 0.9.4 build on May  6 2016 18:01:35.
CTIME 1462842471
USEC 926250
JD 2457518.54712
DATE-OBS 2016-05-10T01:07:51.926
EXPOSURE 0.0
EXPTIME 0.0
INSTRUME 113-03
TELESCOP USNO 1.3m
ORIGIN BNL
NUMBER 2
SCRIPREP 0
SCRIPT for 50 { D 0 }
SCR_COMM 
COMM_NUM 0
CCD_TYPE 
CCD_SER 
BINNING 1x1
BINX 1
BINY 1
SHUTTER 1
DATA_CHANNELS 16
CHAN 16
CHAN1 True
CHAN2 True
CHAN3 True
CHAN4 True
CHAN5 True
CHAN6 True
CHAN7 True
CHAN8 True
CHAN9 True
CHAN10 True
CHAN11 True
CHAN12 True
CHAN13 True
CHAN14 True
CHAN15 True
CHAN16 True
CONFIG /home/ccdtest/reflex/Timing/e2v250/545kHz_20130531a.rcf
RAW_MODE False
baud_rate 9600
int.error_code 0
int.error_source 0
int.error_line 0
int.status_index 142949
int.power 3
int.backplane_type 268566784
int.interface_type 537067776
int.powerA_type 805634304
int.powerB_type 1074004224
int.daughter1_type 1359085824
int.daughter2_type 0
int.daughter3_type 1879245056
int.daughter4_type 1879245056
int.dau

In [141]:
path = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/calibs/'
n_files = 0
for fname in os.listdir(path):
    filename = path + fname
    
    obj = ''
    if fname.find('bias')!=-1:
        obj = 'BIAS'
    elif fname.find('dark')!=-1:
        obj = 'DARK'
    elif fname.find('flat')!=-1:
        obj = 'FLAT'
    else:
        obj = 'error'
        print '%s --> %s'%(fname, obj)
        break
    
    d_file = pf.open(filename, mode='update')
    primaryHeader = d_file[0].header

    print '%s --> %s'%(fname, obj)
    primaryHeader['IMGTYPE'] = obj
    d_file.flush()
    d_file.close()
#     break
    n_files += 1
    if n_files %100 ==0: print 'Fixed %s files'%n_files

print 'Fixed %s files'%n_files

2016-05-10_dark_01.fits --> DARK
2016-05-10_dark_53.fits --> DARK
2016-05-10_mbias_75.fits --> BIAS
2016-05-10_skyflats_40.fits --> FLAT
2016-05-11_mdarks_282.fits --> DARK
2016-05-11_morningflats_38.fits --> FLAT
2016-05-11_mdarks_261.fits --> DARK
2016-05-10_mbias_13.fits --> BIAS
2016-05-11_skyflats_40.fits --> FLAT
2016-05-11_morningflats_45.fits --> FLAT
2016-05-10_bias_18.fits --> BIAS
2016-05-11_mdarks_023.fits --> DARK
2016-05-10_bias_29.fits --> BIAS
2016-05-11_mdarks_048.fits --> DARK
2016-05-10_dark_31.fits --> DARK
2016-05-10_mbias_17.fits --> BIAS
2016-05-11_mdarks_203.fits --> DARK
2016-05-10_mbias_26.fits --> BIAS
2016-05-12_bias_16.fits --> BIAS
2016-05-12_mbias_30.fits --> BIAS
2016-05-10_bias_22.fits --> BIAS
2016-05-11_mdarks_183.fits --> DARK
2016-05-11_mdarks_225.fits --> DARK
2016-05-11_mdarks_306.fits --> DARK
2016-05-11_morningflats_01.fits --> FLAT
2016-05-11_mdarks_195.fits --> DARK
2016-05-11_morningflats_09.fits --> FLAT
2016-05-11_mdarks_150.fits --> DARK
2

In [140]:
fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/calibs/2016-05-10_dark_01.fits', max_lines=999)


SIMPLE True
BITPIX 16
NAXIS 0
EXTEND True
HISTORY Created with RTS2 version 0.9.4 build on May  6 2016 18:01:35.
CTIME 1462843012
USEC 857171
JD 2457518.55338
DATE-OBS 2016-05-10T01:16:52.857
EXPOSURE 1.0
EXPTIME 1.0
INSTRUME 113-03
TELESCOP USNO 1.3m
ORIGIN BNL
NUMBER 57
SCRIPREP 0
SCRIPT for 9 { D 1 D 5 D 10 D 30 D 60 D 120 D 240 }
SCR_COMM 
COMM_NUM 0
CCD_TYPE 
CCD_SER 
BINNING 1x1
BINX 1
BINY 1
SHUTTER 1
DATA_CHANNELS 16
CHAN 16
CHAN1 True
CHAN2 True
CHAN3 True
CHAN4 True
CHAN5 True
CHAN6 True
CHAN7 True
CHAN8 True
CHAN9 True
CHAN10 True
CHAN11 True
CHAN12 True
CHAN13 True
CHAN14 True
CHAN15 True
CHAN16 True
CONFIG /home/ccdtest/reflex/Timing/e2v250/545kHz_20130531a.rcf
RAW_MODE False
baud_rate 9600
int.error_code 0
int.error_source 0
int.error_line 0
int.status_index 143144
int.power 3
int.backplane_type 268566784
int.interface_type 537067776
int.powerA_type 805634304
int.powerB_type 1074004224
int.daughter1_type 1359085824
int.daughter2_type 0
int.daughter3_type 1879245056
int.da

In [188]:
path = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs/'
n_files = 0
for fname in os.listdir(path):
    filename = path + fname
#     print filename
    
    d_file = pf.open(filename)
    data_primaryHeader = d_file[0].header
    date = data_primaryHeader['DATE-OBS']
    
    obj = ''
    if fname.find('bias')!=-1:
        obj = 'BIAS'
    elif fname.find('dark')!=-1:
        obj = 'DARK'
    elif fname.find('flat')!=-1:
        obj = 'FLAT'
    else:
        obj = 'error'
        print '%s --> %s'%(fname, obj)
        break
    del data_primaryHeader
    d_file.close()

    md_filename = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/dummy_' + str(n_files).zfill(4) + '.fits'
    shutil.copy('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/dummy_original',md_filename)
    md_file = pf.open(md_filename, mode='update')
    md_primaryHeader = md_file[0].header
    md_primaryHeader['DATE-OBS'] = date.split('T')[0]
    md_primaryHeader['UTC'] = date.split('T')[1]
    md_primaryHeader['OBJECT'] = obj

    if obj == 'BIAS' or obj == 'DARK':
        md_primaryHeader['FILTER'] = obj
        md_primaryHeader['OBJECT'] = obj

    
    md_file.flush()
    md_file.close()   
    del md_primaryHeader
    n_files += 1
#     if n_files ==3: break
    if n_files %100 ==0: print 'Fixed %s files'%n_files

print 'Fixed %s files'%n_files

Fixed 100 files
Fixed 200 files
Fixed 300 files
Fixed 400 files
Fixed 500 files
Fixed 600 files
Fixed 700 files
Fixed 800 files
Fixed 896 files


In [218]:
fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/dummy_original', max_lines=1005)
# print '----'
# fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3/2016-05-10_CAblank5_i_01.fits', max_lines=20000)

SIMPLE True
BITPIX 16
NAXIS 2
NAXIS1 100
NAXIS2 100
DATE 2016-05-11T02:36:34
ORIGIN USNO-FS
TIMESYS UTC
DATE-OBS 2016-05-11
TELESCOP NO13
INSTRUME ashutter
OBSERVER mam
OBJECT wakeup
IMAGETYP OBJECT
FILTER Y3
RA 11:10:47.48
DEC +30:05:12.5
EQUINOX 2000.0
RAENC 11:11:39.94
DECENC +30:00:00.5
ROTATOR 0.0
UTC 02:36:29.552
ST 10:26:47.854
EXPTIME 2.0007
DARKTIME 2.0007
AIRMASS 1.018
GAIN 5
RDNOISE 5
DATASEC [6:100,1:100]
CCDSEC [1:95,1:100]
BIASSEC [1:5,1:100]
NSUM 1 1
LTM1_1 1
LTM1_2 0
LTM2_1 0
LTM2_2 1
LTV1 5
LTV2 0
DTM1_1 1
DTM1_2 0
DTM2_1 0
DTM2_2 1
DTV1 0
DTV2 0
SCALE 0.96
PRESKIP 0
CCDIDENT 24
CCDFLIP 0
CCDXBIN 1
CCDYBIN 1
CCDGAIN 0
EXEC_VER ???
TEMPS 5 16.48 221.16 13.20 15.35 14.26 0.00 0.00 0.00
COMMENT VU01 UT=02:36:28.70  LST=10:26:46.71  HA=E0:44:53.12  EP=2016.359
VU02 2016d132            DOME    MOVING
VU03         (ZeroPoint) RA=W0:00:00.00   Dec=+00:00:00.0
VU04         (2000.000)  RA=10:15:32.05   Dec=+30:00:00.0
VU05         (Apparent)  RA=10:16:27.55   Dec=+29:55:08.0
VU

In [196]:
n_files = 0
md_files = []
for data_file in pairings:
    md_file = pairings[data_file]
    md_files.append(md_file)
#     d_file = pf.open(data_file, mode='update')
#     m_file = pf.open(md_file)

#     primaryHeader = d_file[0].header
#     md_primaryHeader = m_file[0].header

# #     primaryHeader['DATE-OBS'] = md_primaryHeader['DATE']
# #     primaryHeader['DATE'] = md_primaryHeader['DATE']
#     just_date = md_primaryHeader['DATE-OBS']
#     just_UTC = md_primaryHeader['UTC']
#     new_date_obs = just_date + 'T' + just_UTC
#     primaryHeader['DATE-OBS'] = new_date_obs
    
#     d_file.flush()
#     d_file.close()
#     m_file.close()
# #     break
#     n_files += 1
#     if n_files %100 ==0: print 'Fixed %s files'%n_files

# print 'Finished fixing %s files'%n_files


In [211]:
used_files = []
unused_files = []

for fname in os.listdir('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/'):
    if fname == '..':
        print 'hello'
        break
    filename = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/' + fname
    if filename in md_files:
        used_files.append(fname)
    else:
        unused_files.append(fname)
        

In [212]:
print len(used_files)

998


In [203]:
print len(os.listdir('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/'))

1366


In [214]:
for filename in unused_files:
    shutil.move('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/'+filename, '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/unused_metadata/'+filename)

IOError: [Errno 2] No such file or directory: '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/2016-05-07_g16d128.023'

In [219]:
filters = []
path = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/1m3_metadata/'

n_files = 0
for fname in os.listdir(path):
    filename = path + fname
    d_file = pf.open(filename)
    data_primaryHeader = d_file[0].header
    filt = data_primaryHeader['FILTER']
    
    if not filt in filters:
        filters.append(filt)
    d_file.close()
    n_files += 1
    if n_files %100 ==0: print 'Processed %s files'%n_files
        

Processed 100 files
Processed 200 files
Processed 300 files
Processed 400 files
Processed 500 files
Processed 600 files
Processed 700 files
Processed 800 files
Processed 900 files


In [220]:
for filt in filters:
    print filt

GRATING
X
Y3
Y4
U
Z
SDSSR
SDSSG
SDSSI


In [221]:
filters_61inch = []
path = '/nfs/lsst2/photocalData/data/monocam/sanitised/61inch/61inch_metadata/'

n_files = 0
for fname in os.listdir(path):
    filename = path + fname
    d_file = pf.open(filename)
    data_primaryHeader = d_file[0].header
    filt = data_primaryHeader['FILTER']
    
    if not filt in filters_61inch:
        filters_61inch.append(filt)
    d_file.close()
    n_files += 1
    if n_files %100 ==0: print 'Processed %s files'%n_files
        

Processed 100 files
Processed 200 files
Processed 300 files
Processed 400 files


In [222]:
for filt in filters_61inch:
    print filt

SDSSR
SDSSZ
SDSSI
SDSSG
OPEN


In [226]:
# path = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/'
# n_files = 0
# for fname in os.listdir(path):
#     filename = path + fname
# #     print filename
    
#     d_file = pf.open(filename)
#     data_primaryHeader = d_file[0].header
#     date = data_primaryHeader['DATE-OBS']
    
#     obj = ''
#     if fname.find('bias')!=-1:
#         obj = 'BIAS'
#     elif fname.find('dark')!=-1:
#         obj = 'DARK'
#     elif fname.find('flat')!=-1:
#         obj = 'FLAT'
#     else:
#         obj = 'error'
#         print '%s --> %s'%(fname, obj)
#         break
#     del data_primaryHeader
#     d_file.close()

#     md_filename = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/dummy_' + str(n_files).zfill(4) + '.fits'
#     shutil.copy('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/dummy_original',md_filename)
#     md_file = pf.open(md_filename, mode='update')
#     md_primaryHeader = md_file[0].header
#     md_primaryHeader['DATE-OBS'] = date.split('T')[0]
#     md_primaryHeader['UTC'] = date.split('T')[1]
#     md_primaryHeader['OBJECT'] = obj

#     if obj == 'BIAS' or obj == 'DARK':
#         md_primaryHeader['FILTER'] = obj
#         md_primaryHeader['OBJECT'] = obj

    
#     md_file.flush()
#     md_file.close()   
#     del md_primaryHeader
#     n_files += 1
# #     if n_files ==3: break
#     if n_files %100 ==0: print 'Fixed %s files'%n_files

# print 'Fixed %s files'%n_files





calib_pairings = {}

data_path = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs/'
md_path = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/'

for fname in os.listdir(data_path):
    filename = data_path + fname
#     print filename
    f = pf.open(filename)
    primaryHeader = f[0].header
    header_number = primaryHeader['NUMBER']
    date = primaryHeader['DATE-OBS'].split('T')[0]
    time = primaryHeader['DATE-OBS'].split('T')[1]
    f.close()

    glob_string = md_path + date + '_*.' + str(header_number).zfill(3)
    mdfiles = glob.glob(glob_string)
    if len(mdfiles) != 1:
        print 'Found %s md files for %s'%(len(mdfiles),filename)
        problem_dest = data_path + 'no_matching_md/' + fname
#         shutil.move(filename, problem_dest)
        print
        print

    else:
        calib_pairings[filename] = mdfiles[0]
print 'finished'
print 'Found %s pairings'%len(calib_pairings)



finished
Found 1000 pairings


In [249]:
print len(os.listdir('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs/'))

temp = dict((y,x) for x,y in calib_pairings.iteritems())
print len(temp)
n_in, n_out = 0,0
for fname in os.listdir('/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/'):
    filename = '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/' + fname
#     print filename
#     break
    if filename in temp:
#         print '1'
        n_in +=1
    else:
#         shutil.move(filename, '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/dummyfile')
        n_out +=1
print n_in, n_out

896
657
657 709


In [236]:
print '/nfs/lsst2/photocalData/data/monocam/sanitised/1m3/calibs_dummy_metadata/2016-05-11_g16d132.033' in temp

True


In [246]:
print len(calib_pairings)

896


In [252]:
test = []
for key in calib_pairings:
#     value =calib_pairings[key]
    if value not in test: test.append(key)

In [253]:
print len(test)

896


In [256]:
print len(pairings)
temp = dict((y,x) for x,y in pairings.iteritems())
print len(temp)

1000
998


In [259]:
fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sanitised3/1m3/calibs/2016-05-10bias_01.fits_shutter_dummyfile', max_lines=30)
print '----'
fn.printHeaderData('/nfs/lsst2/photocalData/data/monocam/sanitised3/1m3/calibs/2016-05-10bias_01.fits', max_lines=20)


SIMPLE True
BITPIX 16
NAXIS 2
NAXIS1 100
NAXIS2 100
DATE 2016-05-10T02:32:05.959
ORIGIN USNO-FS
TIMESYS UTC
DATE-OBS 2016-05-10
TELESCOP NO13
INSTRUME ashutter
OBSERVER mam
OBJECT BIAS
IMAGETYP OBJECT
FILTER OPEN
RA 11:02:05.13
DEC +30:05:08.5
EQUINOX 2000.0
RAENC 11:02:58.02
DECENC +29:59:59.1
ROTATOR 0.0
UTC 02:32:05.959
ST 10:18:26.979
EXPTIME 0.0
DARKTIME 2.0007
AIRMASS 1.018
GAIN 5
RDNOISE 5
DATASEC [6:100,1:100]
CCDSEC [1:95,1:100]
----
SIMPLE True
BITPIX 16
NAXIS 0
EXTEND True
HISTORY Created with RTS2 version 0.9.4 build on May  6 2016 18:01:35.
CTIME 1462842471
USEC 926250
JD 2457518.54712
DATE-OBS 2016-05-10T02:32:05.959
EXPOSURE 0.0
EXPTIME 0.0
INSTRUME 113-03
TELESCOP USNO 1.3m
ORIGIN BNL
NUMBER 2
SCRIPREP 0
SCRIPT for 50 { D 0 }
SCR_COMM 
COMM_NUM 0
CCD_TYPE 
